In [1]:
import datasets as ds
import pandas as pd
from itertools import combinations
from rich.progress import track
from matplotlib import pyplot as plt
from rich import print

%matplotlib inline

In [2]:
dataset = ds.load_dataset("Rexhaif/wmt23-24")

In [3]:
def make_pairs(dataset):
    data = dataset.to_pandas()
    result = []
    for src, subset in track(data.groupby('src'), description='Processing'):
        pairs = list(combinations(subset.to_dict(orient='records'), 2))
        for item1, item2 in pairs:
            result.append({
                'lp': item1['lp'],
                'src': item1['src'],
                'ref': item1['ref'],
                'hyp0': item1['hyp'],
                'hyp1': item2['hyp'],
                'score0': item1['score'],
                'score1': item2['score'],
                'system0': item1['system'],
                'system1': item2['system'],
                'score_diff': abs(abs(item1['score']) - abs(item2['score'])),
                'score_name': item1['score_name'],
                'best_hyp': 0 if item1['score'] > item2['score'] else 1,
            })
    return pd.DataFrame(result)


In [4]:
pairs_train = make_pairs(dataset['train'])

Output()

In [5]:
pairs_train = pairs_train[pairs_train.lp != 'en-cs']

In [6]:
pairs_train[pairs_train.score_diff > 15].shape

(368890, 12)

In [7]:
pairs_train = ds.Dataset.from_pandas(pairs_train)


In [8]:
pairs_train = pairs_train.filter(lambda x: x['score_diff'] > 15)

Filter:   0%|          | 0/893383 [00:00<?, ? examples/s]

In [10]:
pairs_train = pairs_train.remove_columns(['__index_level_0__'])

In [11]:

pairs_train.push_to_hub('Rexhaif/wmt23-pairs')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/369 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Rexhaif/wmt23-pairs/commit/ddc2e991f4e5aafde05917067dff894e9587383c', commit_message='Upload dataset', commit_description='', oid='ddc2e991f4e5aafde05917067dff894e9587383c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Rexhaif/wmt23-pairs', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Rexhaif/wmt23-pairs'), pr_revision=None, pr_num=None)